In [ ]:
!pip install earthengine-api

In [1]:
import ee

# Authenticate the Earth Engine library.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


In [2]:
import ee
import folium
from folium.plugins import MarkerCluster
import geemap.core as geemap

m = geemap.Map()
m.set_center(99, 18.80, 8)

In [ ]:
roi = ee.Geometry.Polygon(
    [[[99.0, 18.0],
      [99.0, 19.0],
      [100.0, 19.0],
      [100.0, 18.0],
      [99.0, 18.0]]]
)

print(roi.getInfo())

In [ ]:
# add to feature collection
fc = ee.FeatureCollection([
    ee.Feature(roi)
])

# add name to property
fc = fc.set('name', 'Chiang Mai')

print(fc.getInfo())

m.addLayer(fc, {}, 'roi')

In [24]:
ls9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterDate('2023-11-01', '2024-01-31') 

# try to filter by roi

In [37]:
# Scale factors for radiance data
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

# apply scale factors to the dataset
dataset = ls9.map(apply_scale_factors)

In [38]:
# clip the image with roi
dataset = dataset.map(lambda image: image.clip(roi))

In [ ]:
# True color visualization
vis_true = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

# False color visualization
vis_false = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0.0,
    'max': 0.3,
}

In [ ]:
# add to map
m.add_layer(dataset, vis_true, 'True Color (432)')

In [41]:
# reduces an image collectionmedian to mean, max, min 
median = dataset.median()
mean = dataset.mean()
max = dataset.max()
min = dataset.min()

# try to add to map

In [9]:

m.addLayer(dataset, vis_false, 'False color image')

In [7]:
# NDVI visualization
def addNDVI(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

dataset = dataset.map(addNDVI)
ndvi = dataset.select('NDVI')
ndviParams = {'palette': ['blue', 'white', 'green']}
m.addLayer(ndvi, ndviParams, 'NDVI')

In [68]:
# lst from modis
modis = ee.ImageCollection('MODIS/061/MOD11A1') \
    .filterDate('2023-11-01', '2024-03-31') \
    .select('LST_Day_1km') \
    .map(lambda image: image.clip(roi))

# kavin to celsius
cs = modis.map(lambda image: image.multiply(0.02).subtract(273.15))

print(cs.getInfo())

# visualization parameters
lstParams = {
    min: 35.0,
    max: 40.0,
    'palette':['blue', 'white', 'green'],
}
# add to map
m.addLayer(cs, lstParams, 'MODIS LST')



{'type': 'ImageCollection', 'bands': [], 'version': 1722625265194209, 'id': 'MODIS/061/MOD11A1', 'properties': {'system:is_global': 1}, 'features': [{'type': 'Image', 'bands': [{'id': 'LST_Day_1km', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -273.15, 'max': 1037.5500000000002}, 'dimensions': [182, 122], 'origin': [32832, 8519], 'crs': 'SR-ORG:6974', 'crs_transform': [926.6254331383326, 0, -20015109.355797, 0, -926.6254331391667, 10007554.677903]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[99, 18], [100, 18], [100, 19], [99, 19], [99, 18]]]}, 'system:index': '2023_11_01'}}, {'type': 'Image', 'bands': [{'id': 'LST_Day_1km', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -273.15, 'max': 1037.5500000000002}, 'dimensions': [182, 122], 'origin': [32832, 8519], 'crs': 'SR-ORG:6974', 'crs_transform': [926.6254331383326, 0, -20015109.355797, 0, -926.6254331391667, 10007554.677903]}], 'properties': {'system:footprint': {'typ

In [ ]:
# show the map
m